## Environment setup

In [1]:
# import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import os
import folium
import json

In [2]:
# have matplotlib visuals appear in the notebook 

%matplotlib inline

In [9]:
# import ".json" file for counties

county = r'/Users/Cel/Documents/Data Analytics/6. Maine Fisheries Landings/02 Data/geo_maine_counties_no_brackets.geojson'

In [10]:
# check json file contents

f = open(r'/Users/Cel/Documents/Data Analytics/6. Maine Fisheries Landings/02 Data/geo_maine_counties_no_brackets.geojson',)
  
# returns JSON object asa dictionary
data = json.load(f)
  
# Iterating through the json list
for i in data['features']:
    print(i)

{'type': 'Feature', 'geometry': {'coordinates': [[[-69.409635, 44.32799799905699], [-69.41328, 44.32441999905699], [-69.416429, 44.321301999056985], [-69.423443, 44.32238399905699], [-69.42823, 44.32311299905699], [-69.435512, 44.32424699905699], [-69.436988, 44.32448499905699], [-69.43927, 44.32280999905699], [-69.443289, 44.31981499905699], [-69.447484, 44.316709999057], [-69.451251, 44.313949999056994], [-69.450395, 44.311539999057004], [-69.447511, 44.303601999057], [-69.446101, 44.29961899905701], [-69.444218, 44.29438699905702], [-69.442931, 44.29077199905701], [-69.441591, 44.28694199905702], [-69.440665, 44.28443399905702], [-69.44291, 44.27877999905702], [-69.445169, 44.27312699905703], [-69.447301, 44.26777099905703], [-69.449481, 44.26226999905703], [-69.44696, 44.25698599905704], [-69.446062, 44.25506199905704], [-69.445551, 44.25400399905704], [-69.443732, 44.25007499905705], [-69.442427, 44.24718799905703], [-69.44103, 44.24430599905705], [-69.439736, 44.24145699905704], 

In [5]:
# define path
path = '/Users/Cel/Documents/Data Analytics/6. Maine Fisheries Landings'

In [6]:
# import data
df = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'landings_clean.csv'), index_col = False)

## Data cleaning

Consistency checks, duplicates, and missing values were checked in exercise 6.1

In [16]:
df.columns

Index(['Unnamed: 0', 'year', 'species', 'port', 'county', 'weight_type',
       'weight', 'value', 'number_harvesters'],
      dtype='object')

In [17]:
# create a new 'price_per_lb' column
df['price_per_lb'] = df['value']/df['weight']

In [19]:
# Finding extreme 'price_per_lb' values
sns.histplot(df['price_per_lb'], bins=10, kde = True)

<Axes: xlabel='price_per_lb', ylabel='Count'>

In [21]:
# Check how many extreme values

df[df['price_per_lb'] > 2500]

Unnamed: 0  year species            port      county  weight_type  \
3356        3356  2018   Elver         Addison  Washington  Live Pounds   
3358        3358  2018   Elver      Bar Harbor     Hancock  Live Pounds   
3362        3362  2018   Elver         Bristol     Lincoln  Live Pounds   
3365        3365  2018   Elver       Bucksport     Hancock  Live Pounds   
3369        3369  2018   Elver     Dennysville  Washington  Live Pounds   
3383        3383  2018   Elver    Mount Desert     Hancock  Live Pounds   
3397        3397  2018   Elver     Scarborough  Cumberland  Live Pounds   
3401        3401  2018   Elver  South Portland  Cumberland  Live Pounds   
3402        3402  2018   Elver         Steuben  Washington  Live Pounds   
3403        3403  2018   Elver        Sullivan     Hancock  Live Pounds   
3407        3407  2018   Elver           Union        Knox  Live Pounds   
3411        3411  2018   Elver   Winter Harbor     Hancock  Live Pounds   

      weight      value  number_harvesters  price_per_lb  
3356   68.08  170541.98                 33   2505.023208  
3358  124.83  314380.10                 36   2518.465914  
3362   15.17   38136.00                 19   2513.909031  
3365    2.93    7413.50                  4   2530.204778  
3369   20.22   50643.50                  6   2504.624135  
3383   23.82   60076.20                 16   2522.090680  
3397   22.93   57641.90                 33   2513.820323  
3401    2.25    6021.10                  9   2676.044444  
3402  207.63  530184.75                 25   2553.507441  
3403  172.63  440748.55                 38   2553.139952  
3407   41.18  105165.50                 17   2553.800389  
3411   75.96  190332.52                 48   2505.694049

Looked into this fishery last exercise, prices over $2500/lb are normal for elver fishery!

In [23]:
# Finding extreme 'weight' values
sns.histplot(df['weight'], bins=20, kde = True)

<Axes: xlabel='weight', ylabel='Count'>

## Chloropleth

In [7]:
# create subset to plot
data_to_plot = df[['county','value']]
data_to_plot.head()

county      value
0  Washington  208982.04
1     Hancock   15276.99
2     Hancock     448.58
3   Sagadahoc  101804.90
4  Washington   18989.88

In [11]:
# Setup a folium map at a high-level zoom
map = folium.Map(location = [100, 0], zoom_start = 1.5)

# Choropleth maps bind Pandas Data Frames and json geometries.This allows us to quickly visualize data combinations
folium.Choropleth(
    geo_data = county, 
    data = data_to_plot,
    columns = ['county', 'value'],
    key_on = 'feature.properties.coty_name', # this part is very important - check your json file to see where the KEY is located
    fill_color = 'YlOrBr', fill_opacity=0.6, line_opacity=0.1,
    legend_name = "value").add_to(map)
folium.LayerControl().add_to(map)

map

In [12]:
map.save('county_value_data.html')

This choropleth helps answer my research question regarding which county generates the most fishery income to the state of Maine. The answer according to this graphic would be Hancock county, by a landslide! This then leads to more questions around how the catch between counties changes.

In [24]:
# exporting clean data frame
df.to_csv(os.path.join(path, '02 Data','Prepared Data', 'landings_clean.csv'))